In [1]:
import os
import gc
import cv2
import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf
from tqdm import tqdm # To read in images in batches and see progress
from tensorflow import keras
from sklearn.model_selection import train_test_split # For the creation of training and validation sets
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D,MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint 
from keras.preprocessing.image import ImageDataGenerator  #Used for Data augmentation
from keras import backend as K   #For specialized and optimized tensor manipulation
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [3]:
def create_tag_mapping(labels_df):
    labels = set()
    for i in range(len(labels_df)):
        # convert spaced separated tags into an array of tags
        tags = labels_df['tags'][i].split(' ')
        # add tags to the set of known labels
        labels.update(tags)
    # convert set of labels to a list to list
    labels = list(labels)
    labels.sort()
 # dict that maps labels to integers, and the reverse
    labels_map = { labels [i]:i for i in range(len(labels))}
    inv_labels_map = {i: labels [i] for i in range(len(labels))}
    return labels_map, inv_labels_map



def create_file_mapping(train_data):
    mapping = dict()
    for i in range(len(labels_df)):
        name, tags = train_data['image_name'][i], labels_df['tags'][i]
        mapping[name] = tags.split(' ')   
    return mapping


def one_hot_encode(tags,mapping):
    # create empty vector
    encoding = np.zeros(len(mapping), dtype='uint8')
    # mark 1 for each tag in the vector
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding


def load_dataset(path, file_mapping, tag_mapping):
    photos, targets = list(), list()
    # enumerate files in the directory
    for filename in os.listdir(folder):
        # load image
        photo = load_img(path + filename, target_size=(64,64))
        # convert to numpy array
        photo = img_to_array(photo, dtype='uint8')
        # get tags
        tags = file_mapping[filename[:-4]]
        # one hot encode tags
        target = one_hot_encode(tags, tag_mapping)
        # store
        photos.append(photo)
        targets.append(target)
    X = np.asarray(photos, dtype='uint8')
    y = np.asarray(targets, dtype='uint8')
    return X, y
    del photo
    del targets

In [4]:
from keras import backend

def fbeta_score(y_true, y_pred, beta=2):
    # clip predictions
    y_pred = backend.clip(y_pred, 0, 1)
    # calculate elements
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
    # calculate precision
    p = tp / (tp + fp + backend.epsilon())
    # calculate recall
    r = tp / (tp + fn + backend.epsilon())
    # calculate fbeta, averaged across each class
    bb = beta ** 2
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score

In [8]:
filename = '../input/planets-dataset/planet/planet/train_classes.csv'
labels_df = pd.read_csv(filename)

# create a mapping of tags to integers
mapping, inv_mapping = create_tag_mapping(labels_df)
print(len(mapping))

# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(labels_df)

# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(labels_df)

# load the jpeg images
folder = '../input/planets-dataset/planet/planet/train-jpg/'
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)

In [9]:
unique_labels = set()
def append_labels(tags):
    for tag in tags.split():
        unique_labels.add(tag)

train_classes = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
train_classes['tags'].apply(append_labels)
unique_labels = list(unique_labels)

In [15]:

gc.collect()

In [16]:
test_folder = '../input/planets-dataset/planet/planet/'
test = test_folder + 'sample_submission.csv'
test_df = pd.read_csv(test)

x_test = []

test_img_folder = '../input/planets-dataset/planet/planet/test-jpg'
test_img_names = os.listdir(test_img_folder)

n_test = len(test_img_names)
test_classes = test_df.iloc[:n_test, :]
add_classes = test_df.iloc[n_test:, :]


test_img_add_folder = '../input/planets-dataset/test-jpg-additional/test-jpg-additional'
test_add_img_names = os.listdir(test_img_add_folder)

for img_name, _ in tqdm(test_classes.values, miniters=1000):
    img = cv2.resize(cv2.imread(test_img_folder + '/{}.jpg'.format(img_name)), (64, 64))
    x_test.append(img)
    
for img_name, _ in tqdm(add_classes.values, miniters=1000):
    img = cv2.imread(test_img_add_folder + '/{}.jpg'.format(img_name))
    x_test.append(cv2.resize(img, (64, 64)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)

In [ ]:

gc.collect()

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True,random_state=1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

## **Building Model Architecture**

To tackle this multi-label problem, a combination of a custom deep CNN architecture along with the pre-trained CNN architecture(VGG16) was implemented in Keras with Tensorflow backend

In [23]:
input_size = 64
channels = 3

model = Sequential()

#input layer
model.add(BatchNormalization(input_shape=(input_size,input_size,channels)))

#CCM_1
model.add(Conv2D(32,kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#CCM_2
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#CCM_3
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#CCM_4
model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Create a feature vector from the CCM_4 final layer
model.add(Flatten())

# Fully Connected (FC) Layer
model.add(Dense(512, activation='relu'))
model .add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(17, activation='sigmoid'))


In [24]:
out_shape=17
import tensorflow.keras as keras
# Loading the pre-trained VGG16 architecture module
from tensorflow.keras.applications.vgg16 import VGG16



# Extract the pre - trained architecture
base_model = VGG16(input_shape =(input_size,input_size,3),include_top =False,weights ='imagenet')
base_model.summary()

# Get the output of the base_model formed above
x = base_model.output
# Flatten to obtain a feature vector
x = Flatten()(x)
# Connect the feature vector to to the fully connected (FC) layer
x = Dense (512 , activation ='relu')(x)
# Form the output label predictions
predictions = Dense (out_shape, activation ='sigmoid')(x)
model = Model(inputs= base_model.input,outputs = predictions)

In [25]:
gc.collect()

In [26]:
# Implementing ImageDataGenerator for data augmentation. This is an important technique which reduces 
# overfitting as it generates extra images by flipping, cropping, zooming e,t.c the images. This makes 
# the model have more images to learn from.

train_datagen = ImageDataGenerator(featurewise_center=True, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   rotation_range=90,
                                   zoom_range=0.2,
                                  fill_mode ='reflect'
                                  )
test_datagen = ImageDataGenerator(featurewise_center=True)


In [27]:
train_datagen.fit(X_train)
test_datagen.fit(X_test)
train_it = train_datagen.flow(X_train, y_train, batch_size=128)
test_it = test_datagen.flow(X_test, y_test, batch_size=128)

In [35]:
gc.collect()

In [29]:
# Defining other parameters
epochs=20 # An epoch is one complete pass through the training data, We specify 20 here
optimizer = keras.optimizers.Adam(learning_rate=0.0001) # Defining our Adam optimizer and learning rate

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=[fbeta_score])

callback = [EarlyStopping(monitor='val_loss',patience=2,verbose=0),
             ModelCheckpoint(filepath='weights/best_weights',save_best_only=True,save_weights_only=True)
            ]

In [32]:
model.fit_generator(train_it,
                    steps_per_epoch = len(train_it),
                    validation_data = (test_it),
                    validation_steps =len(test_it),
                    epochs =epochs,
                    callbacks = callback
                   )

In [39]:
gc.collect()

In [ ]:

test_ = []
test_.append (model.predict (x_test, batch_size = 128, verbose = 2))
result = np.array(test_[0])
for i in range(1,len(test_)):
    result += np.array(test_)
result = pd.DataFrame(result,columns = unique_labels)

pred = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    pred.append(' '.join(list(a.index)))

In [41]:
gc.collect()

In [44]:
test_df['tags'] = pred
test_df.to_csv('amazon.csv', index=False) 